In [1]:
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [33]:
in_file = "../features/UCM-ResNet-152.csv"
out_file = "../results/" + in_file.split("/")[-1].split(".csv")[0] + ".npy"
df = pd.read_csv("../features/UCM-ResNet-152.csv")

In [3]:
df.head()

Unnamed: 0         0         1         2         3         4  \
0        overpass42.tif -3.752808 -0.461312  6.960607  0.093534  3.216339   
1  mobilehomepark75.tif -4.689972 -1.533276 -0.363770 -3.712684 -2.175694   
2     tenniscourt46.tif -5.380649  2.026206 -1.288875 -2.794645 -0.624042   
3        overpass29.tif -5.712953 -1.040280 -2.517505 -4.440084 -2.735700   
4       chaparral41.tif -4.463096 -1.501895 -3.866307 -2.188615 -0.698318   

          5         6         7         8    ...          990       991  \
0  4.002225  3.377012 -0.725385 -0.054136    ...    -3.938607 -3.955094   
1 -1.606550  0.039839 -0.344135 -1.551318    ...    -0.637004 -1.235481   
2 -2.003144 -0.835236  1.924311 -0.835306    ...    -3.942154  1.403567   
3 -2.535528 -3.655354 -1.255664 -0.288036    ...    -4.785943 -4.641560   
4  6.733519  4.000729 -3.779526 -0.615270    ...     1.896740  4.526745   

        992       993       994       995       996       997       998  \
0 -4.418676 -6.199239 -6.088099 -5.453274 -2.186284 -4.251893  0.395046   
1 -3.377314 -2.801301  0.275908 -2.032623  1.609835 -2.581015 -0.597632   
2 -4.646072 -4.062453 -3.283334 -2.077019 -1.828276 -3.919093 -1.109062   
3 -2.772527 -4.608302 -3.955713 -4.166010 -0.935464 -4.376791  0.616531   
4 -2.148957 -2.937480  4.451448  0.490672 -0.508413 -1.133084  0.980089   

        999  
0  2.857523  
1  1.511363  
2  1.274806  
3  2.408259  
4  3.013349  

[5 rows x 1001 columns]

In [4]:
X = np.array(df.iloc[:,1:])

import re
def getdigit(text):
    return re.compile('\d+').findall(text)
labels = [i.split(getdigit(i)[0])[0] for i in df.iloc[:,0]]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels)

print "feature data is shape", X.shape
print "label data is shape", y.shape

feature data is shape (2100, 1000)
label data is shape (2100,)


In [21]:
pipe_svc = Pipeline([('sc1', StandardScaler()),
                    ('clf', SVC(kernel='linear'))])

param_range = np.logspace(-10, 1, 100)
param_grid = [{'clf__C': param_range}]
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, cv=10, n_jobs=-1)

In [22]:
skf = StratifiedKFold(y, n_folds=5)


In [23]:
scores = []
for train_index, test_index in skf:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gs.fit(X_train, y_train)
    clf = SVC(kernel= 'linear', C = gs.best_params_.values()[0])
    clf.fit(X_train, y_train) 
    scores.append(clf.score(X_test, y_test))

In [24]:
scores

[0.95476190476190481,
 0.94761904761904758,
 0.94999999999999996,
 0.93095238095238098,
 0.95952380952380956]

In [25]:
np.mean(scores)

0.94857142857142862

In [26]:
gs.best_params_.values()[0]

0.0021544346900318821

In [35]:
np.save(out_file, scores)